# Capítulo 7 - Pré Processamento de Dados

In [125]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import janitor as jn
from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import (
    ensemble,
    impute,
    model_selection,    
    preprocessing,
    tree,
)

In [126]:
url = ("https://hbiostat.org/data/repo/titanic3.csv")
df = pd.read_csv(url)

## Padronização de Dados

In [127]:
X2 = pd.DataFrame(
    {
        "a": range(5),
        "b": [-100, -50, 0, 200, 1000]
    }
)
X2

,a,b
0,0,-100
1,1,-50
2,2,0
3,3,200
4,4,1000


O `sklearn` possuí um método `.fit_transform()` que combina `.fit()` e `.transform()` em um único método para facilitar o pré-processamento de dados.

O método `fit` é usado para calcular os parâmetros de ajuste de um modelo, enquanto o método `transform` é usado para aplicar os parâmetros de ajuste em um modelo.

Os parâmetros de ajuste são usados para padronizar os dados de entrada.

In [128]:
std = preprocessing.StandardScaler()
std.fit_transform(X2)

array([[-1.41421356, -0.75995002],
       [-0.70710678, -0.63737744],
       [ 0.        , -0.51480485],
       [ 0.70710678, -0.02451452],
       [ 1.41421356,  1.93664683]])

In [129]:
print(f'O desvio padrão é {std.scale_}\nA média é {std.mean_}\ns valores são {std.var_}')

O desvio padrão é [  1.41421356 407.92156109]
A média é [  2. 210.]
s valores são [2.000e+00 1.664e+05]


É possível padronizar usando `pandas`

In [130]:
X_std = (X2 - X2.mean())/ X2.std()
X_std

,a,b
0,-1.264911,-0.679720
1,-0.632456,-0.570088
2,0.000000,-0.460455
3,0.632456,-0.021926
4,1.264911,1.732190


In [131]:
X_std.mean()

a    4.440892e-17
b    0.000000e+00
dtype: float64

In [132]:
X_std.std()

a    1.0
b    1.0
dtype: float64

É possível utilizar a biblioteca `fastai` para padronizar os dados.

In [133]:
from fastai.tabular.all import *

In [134]:
X3 = X2.copy()

count_names = ['a', 'b']

dls = TabularDataLoaders.from_df(X3, procs=[Normalize], cont_names=count_names)
dls.train_ds.conts

,a,b
1,-1.341641,-0.800022
2,-0.447214,-0.681500
3,0.447214,-0.207413
4,1.341641,1.688935


## Escalonamento de Dados

É possível utilizar o método `MinMaxScaler` para escalonar os dados.

In [135]:
from sklearn import preprocessing

In [136]:
mms = preprocessing.MinMaxScaler()
mms.fit_transform(X2)

array([[0.        , 0.        ],
       [0.25      , 0.04545455],
       [0.5       , 0.09090909],
       [0.75      , 0.27272727],
       [1.        , 1.        ]])

Também é possível escaloar os dados utilizando `pandas`.

In [137]:
(X2 - X2.min()) / (X2.max() - X2.min())

,a,b
0,0.00,0.000000
1,0.25,0.045455
2,0.50,0.090909
3,0.75,0.272727
4,1.00,1.000000


## Variáveis Dummy

In [138]:
X_cat = pd.DataFrame(
    {
        "nome": ["Paulo", "Luana"],
        "pais": ["Alemanha", "Brasil"],
    }
)
X_cat

,nome,pais
0,Paulo,Alemanha
1,Luana,Brasil


In [139]:
pd.get_dummies(X_cat)

,nome_Luana,nome_Paulo,pais_Alemanha,pais_Brasil
0,False,True,True,False
1,True,False,False,True


In [140]:
pd.get_dummies(X_cat, drop_first=True) # drop_first=True para evitar a multicolinearidade

,nome_Paulo,pais_Brasil
0,True,False
1,False,True


Também é possível utilizar `pyjanitor` para criar variáveis dummy.

In [141]:
X_cat2 = pd.DataFrame(
    {
        "A": [1, 2, None],
        "nome": ["Paulo", "Luana", "João"],
    }
)
X_cat2

,A,nome
0,1.0,Paulo
1,2.0,Luana
2,NaN,João


In [142]:
jn.expand_column(X_cat2, "nome", sep=",")

,A,nome,João,Luana,Paulo
0,1.0,Paulo,0,0,1
1,2.0,Luana,0,1,0
2,NaN,João,1,0,0


## Codificador de Rótulos

In [143]:
from sklearn import preprocessing

In [144]:
lab = preprocessing.LabelEncoder()
lab.fit_transform(X_cat.nome)

array([1, 0])

Para decodificar os rótulos, é possível utilizar o método `inverse_transform()`.

In [145]:
lab.inverse_transform([1,0])

array(['Paulo', 'Luana'], dtype=object)

O `pandas` também possui um método para codificar rótulos.

Deve-se converter a coluna em um tipo de dados categórico antes de codificar os rótulos, em seguida extrair o código númerico.

Dessa forma, cria-se uma nova coluna com os rótulos codificados. Aconselhavel o uso do `.as_ordered()` para manter a ordem dos rótulos.

In [146]:
y = X_cat.nome.astype(
    "category"
).cat.as_ordered().cat.codes # cat.codes é o mesmo que LabelEncoder

In [147]:
print(f'X_cat\n{X_cat}\n\ny\n{y}')

X_cat
    nome      pais
0  Paulo  Alemanha
1  Luana    Brasil

y
0    1
1    0
dtype: int8


## Codificação de frequência

In [148]:
mapping = X_cat.nome.value_counts()
X_cat.nome.map(mapping)

0    1
1    1
Name: nome, dtype: int64

## Extraindo categorias a partir de strings

In [149]:
from collections import Counter

In [150]:
c = Counter()

def triples(val):
    for i in range(len(val)):
        c[val[i : i + 3]] += 1

df.name.apply(triples)

0       None
1       None
2       None
3       None
4       None
        ... 
1304    None
1305    None
1306    None
1307    None
1308    None
Name: name, Length: 1309, dtype: object

In [151]:
c.most_common(10)

[(', M', 1282),
 (' Mr', 954),
 ('r. ', 830),
 ('Mr.', 757),
 ('s. ', 460),
 ('n, ', 320),
 (' Mi', 283),
 ('iss', 261),
 ('ss.', 261),
 ('Mis', 260)]

É possível usar expressões regulares para extrair categorias a partir de strings.

In [152]:
df.name.str.extract(
    "([A-Za-z]+)\\.", expand=False
).head()

0      Miss
1    Master
2      Miss
3        Mr
4       Mrs
Name: name, dtype: object

Utilizando o método `value_counts()` é possível verificar a frequência de cada categoria.

In [153]:
df.name.str.extract(
    "([A-Za-z]+)\\.", expand=False
).value_counts()

name
Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Ms            2
Major         2
Capt          1
Sir           1
Dona          1
Jonkheer      1
Countess      1
Don           1
Mme           1
Lady          1
Name: count, dtype: int64

## Outras codificações

In [154]:
import category_encoders as ce

In [155]:
X_cat

,nome,pais
0,Paulo,Alemanha
1,Luana,Brasil


Explicação do `HashingEncoder`:

A técnica de hashing é uma técnica de codificação que mapeia valores de entrada arbitrários para valores de saída de comprimento fixo. Esses valores são binários.

A `col_0` Chamada de `intercept column` é uma que, se o valor for 1, indica que a categoria original estava presente. Se o valor for 0, indica que a categoria original não estava presente.

A `col_n` é a coluna para evitar colisões de hash.

Cada outra `col` é uma coluna de saída para uma categoria

In [156]:
he = ce.HashingEncoder(verbose=1)
he.fit_transform(X_cat)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,0,1,0,0,1,0,0,0
1,0,0,0,2,0,0,0,0


Também pode ser utilizado do codificador de ordinais para converter valores com ordem em números. Se houver um valor ausente na coluna, o valor default é `-1` para o valor ausente.

In [157]:
size_df = pd.DataFrame(
    {
        "size": ["S", "M", "L", "XL", np.nan],
        "name": ["Paulo", "Luana", "João", "Maria", "José"],
    }
)
size_df

,size,name
0,S,Paulo
1,M,Luana
2,L,João
3,XL,Maria
4,NaN,José


In [158]:
ore = ce.OrdinalEncoder(
    mapping= [
        {
            "col": "size",
            "mapping": {
                "S": 1,
                "M": 2,
                "L": 3,
                "XL": 4,
                "XG": 5,
            }
        }
    ]
)

In [159]:
ore.fit_transform(size_df)

,size,name
0,1.0,Paulo
1,2.0,Luana
2,3.0,João
3,4.0,Maria
4,-1.0,José


Se tiver dados com alta cardinalidade(muitos valores únicos), é aconselhável usar os codificadores bayesianos. São eles: `TargetEncoder`, `LeaveOneOutEncoder`, `WeightOfEvidenceEncoder`, `JamesSteinEncoder`, `MEstimateEncoder`, `CatBoostEncoder`.

Por exemplo, convertendo a coluna `survived` em uma coluna de probabilidade de sobrevivência.

In [160]:
def get_title(df):
    return df.name.str.extract(
        "([A-Za-z]+)\\.", expand=False
    )
te = ce.TargetEncoder(cols="Title")
te.fit_transform(
    df.assign(Title=get_title), df.survived
)["Title"].head()

0    0.676923
1    0.506139
2    0.676923
3    0.162483
4    0.786802
Name: Title, dtype: float64

## Engenharia de Dados para Datas


In [164]:
from fastai.tabular.all import *

In [177]:
data = pd.DataFrame(
    {
        "A": pd.to_datetime(
            ["10/23/2023", "10/24/2023"],
            format="%m/%d/%Y",
            errors="coerce"
        ),
    }
)

data

,A
0,2023-10-23
1,2023-10-24


In [181]:
add_datepart(data, "A")

,AYear,AMonth,AWeek,ADay,ADayofweek,ADayofyear,AIs_month_end,AIs_month_start,AIs_quarter_end,AIs_quarter_start,AIs_year_end,AIs_year_start,AElapsed
0,2023,10,43,23,0,296,False,False,False,False,False,False,1.698019e+09
1,2023,10,43,24,1,297,False,False,False,False,False,False,1.698106e+09


## Adição do atributo col_na

In [195]:
from pandas.api.types import is_numeric_dtype
def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (
            name in na_dict
        ):
            df[name + "_na(null)"] = pd.isnull(col)
            filler = (
                na_dict[name]
                if name in na_dict
                else col.median()
            )
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

In [196]:
data = pd.DataFrame(
    {
        "A": [1, 4, None],
    }
)

data

,A
0,1.0
1,4.0
2,NaN


In [197]:
fix_missing(data, data.A, "A", {})

{'A': 2.5}

In [198]:
data

,A,A_na(null)
0,1.0,False
1,4.0,False
2,2.5,True


Ou uma versão mais simples:

In [200]:
data = pd.DataFrame(
    {
        "A": [1, 4, None],
    }
)

data["A_na(null)"] = pd.isnull(data.A)
data["A"] = data.A.fillna(data.A.median())
data

,A,A_na(null)
0,1.0,False
1,4.0,False
2,2.5,True


## Engenharia de dados manual

In [209]:
numeric_cols = df.select_dtypes(include='number').columns
agg = (
    df.groupby("cabin")[numeric_cols]
    .agg(['min', 'max', 'mean', 'sum'])
    .reset_index()
)

agg.columns = [f'{col[0]}_{col[1]}' if col[1] else f'{col[0]}' for col in agg.columns]

agg_df = df.merge(agg, on="cabin")

print(agg_df)

      pclass  survived                               name     sex   age  \
0          1         1      Allen, Miss. Elisabeth Walton  female  29.0   
1          1         0       Allison, Miss. Helen Loraine  female   2.0   
2          1         1  Andrews, Miss. Kornelia Theodosia  female  63.0   
3          1         1       Barber, Miss. Ellen "Nellie"  female  26.0   
4          1         1              Bazzani, Miss. Albina  female  32.0   
...      ...       ...                                ...     ...   ...   
1304       2         0             Lahtinen, Rev. William    male  30.0   
1305       2         0              Montvila, Rev. Juozas    male  27.0   
1306       2         0      Peruschitz, Rev. Joseph Maria    male  41.0   
1307       2         1          Reynaldo, Ms. Encarnacion  female  28.0   
1308       3         0            O'Donoghue, Ms. Bridget  female   NaN   

      sibsp  parch  ticket      fare cabin  ... parch_mean parch_sum  \
0         0      0   24160 